In [1]:
# library imports
import numpy as np
import astropy
from astropy.coordinates import SkyCoord, GCRS, ICRS, HeliocentricMeanEcliptic, EarthLocation
from astropy.units import deg, meter

#  MSE imports
import astro_utils
import asteroid_integrate
import asteroid_data
from asteroid_data import make_data_one_file, get_earth_pos

In [2]:
# RA and DEC of Ceres at 58600.5 according to JPL
ra_num = 252.25008
dec_num = -17.00954
r_num = 1.868043

# MJD of observation time
obstime_mjd = 58600.5

# Reference frames
# https://docs.astropy.org/en/stable/api/astropy.coordinates.GCRS.html
# https://docs.astropy.org/en/stable/api/astropy.coordinates.ICRS.html
# frame = ICRS
frame_earth = ICRS
frame_solar = HeliocentricMeanEcliptic

In [3]:
# Earth location of the Palomar observatory
geoloc_palomar_auto = EarthLocation.of_site('Palomar')
print(f'geolocation of Palomar observatory:')
print(f'geoloc_palomar = {geoloc_palomar_auto}')
print(f'geoloc_palomar geodetic = {geoloc_palomar_auto.geodetic}')

geolocation of Palomar observatory:
geoloc_palomar = (-2410346.78217658, -4758666.82504051, 3487942.97502457) m
geoloc_palomar geodetic = GeodeticLocation(lon=<Longitude -116.863 deg>, lat=<Latitude 33.356 deg>, height=<Quantity 1706. m>)


In [4]:
# For some reason, the 'nice' way to do this below leads to a bug in constructor to GCRS
# geoloc_palomar = EarthLocation(-2410346.8, -4758666.8, 3487943, unit='m')

# Workaround: extract the geolocation of the Palomar observatory using EarthLocation.of_site
geoloc_palomar = [-2410346.78217658, -4758666.82504051, 3487942.97502457] * meter

**Coordinates of Ceres at MJD 58600.5 as SkyCoord instance**

In [5]:
c = SkyCoord(ra=ra_num, dec=dec_num, unit='deg', frame=frame_earth)

In [6]:
c = SkyCoord(ra=ra_num*astropy.units.deg, dec=dec_num*astropy.units.deg, distance=r_num*astropy.units.au, frame=frame_earth)

In [7]:
c = SkyCoord(ra=ra_num, dec=dec_num, distance=r_num, unit='deg', frame=frame_earth)

In [8]:
# switch from numbers to class instances
ra = ra_num * astropy.units.deg
dec = dec_num * astropy.units.deg
distance = r_num * astropy.units.au

# distances of zero and one AU
zero_au = 0.0 * astropy.units.au 
one_au = 1.0 * astropy.units.au

# angle of zero degrees
zero_deg = 0.0 * astropy.units.deg

# observation time
obstime = astropy.time.Time(val=obstime_mjd, format='mjd')

# display class outputs
print(f'ra = {ra}')
print(f'dec = {dec}')
print(f'distance = {distance}')
print(f'obstime = {repr(obstime)}')

ra = 252.25008 deg
dec = -17.00954 deg
distance = 1.868043 AU
obstime = <Time object: scale='utc' format='mjd' value=58600.5>


In [9]:
# Coordinates of Ceres in geocentric frame; include observation time & location.  
# Use correct distance of 1.868 AU from JPL
ast_geo = GCRS(ra=ra, dec=dec, distance=distance, obstime=obstime, obsgeoloc=geoloc_palomar)
ast_geo

<GCRS Coordinate (obstime=58600.5, obsgeoloc=(-2410346.78217658, -4758666.82504051, 3487942.97502457) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, AU)
    (252.25008, -17.00954, 1.868043)>

In [10]:
# Coordinates of Earth in geocentric frame; this is easy, distance is zero!
earth_geo = GCRS(ra=zero_deg, dec=zero_deg, distance=zero_au, obstime=obstime, obsgeoloc = geoloc_palomar)
earth_geo

<GCRS Coordinate (obstime=58600.5, obsgeoloc=(-2410346.78217658, -4758666.82504051, 3487942.97502457) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, AU)
    (0., 0., 0.)>

**Unit direction of Ceres in ICRS (J2000) Frame**

In [11]:
# Relative displacement from Earth to Ceres in the Heliocentric frame
# This is critical, need to transform from ICRS to Heliocentric for axes to be oriented correctly
earth_hel = earth_geo.transform_to(frame_solar)
ast_hel = ast_geo.transform_to(frame_solar)
q_rel_jpl = (ast_hel.cartesian - earth_hel.cartesian).xyz.value
print(f'q_earth_hel = {earth_hel.cartesian}')
print(f'q_ast_hel =   {ast_hel.cartesian}')
print(f'q_rel_jpl =   {q_rel_jpl}')

q_earth_hel = (-0.80169347, -0.5909504, -0.00017422) AU
q_ast_hel =   (-1.34641857, -2.36918041, 0.17520326) AU
q_rel_jpl =   [-0.54472509 -1.77823001  0.17537748]


In [12]:
# Direction from earth to asteroid
u_jpl = q_rel_jpl / np.linalg.norm(q_rel_jpl)
np.round(u_jpl, 6)

array([-0.291602, -0.951921,  0.093883])

In [13]:
# Calculation in a single step using coordinate transformations.
# don't need the distance, just the RA and the Dec!
# q_ast_jpl = SkyCoord(ra=ra, dec=dec, distance=distance, obstime=obstime, obsgeoloc=geoloc_palomar, frame=frame_geo)
q_ast_jpl = SkyCoord(ra=ra, dec=dec, distance=one_au, obstime=obstime, frame=frame_earth)
u_jpl = q_ast_jpl.transform_to(frame_solar).cartesian.xyz.value
u_jpl = u_jpl / np.linalg.norm(u_jpl)
np.round(u_jpl, 6)

array([-0.287882, -0.953112,  0.093281])

In [14]:
# check that this has norm 1 as expected
np.linalg.norm(u_jpl)

1.0

**MSE Integrated Coordinates of Ceres at 58600.5**

In [15]:
ast_elt = asteroid_integrate.load_data()

In [16]:
inputs, outputs = make_data_one_file(0, 1000)

In [17]:
inputs.keys()

dict_keys(['a', 'e', 'inc', 'Omega', 'omega', 'f', 'epoch', 'ts'])

In [18]:
outputs.keys()

dict_keys(['q', 'v', 'u'])

In [19]:
astro_utils.mjd_to_date(int(obstime_mjd))

datetime.date(2019, 4, 27)

In [20]:
astro_utils.mjd_to_datetime(obstime_mjd)

datetime.datetime(2019, 4, 27, 12, 0)

In [21]:
ast_elt

,Num,Name,epoch_mjd,a,e,inc,Omega,omega,M,H,G,Ref,f,P,n,long,theta,pomega,T_peri
Num,,,,,,,,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.34,0.12,JPL 46,1.501306,1683.145749,0.003733,4.036516,4.187424,2.686118,-361.745873
2,2,Pallas,58600.0,2.772466,0.230337,0.608007,3.020817,5.411373,1.041946,4.13,0.11,JPL 35,1.490912,1686.155979,0.003726,3.190951,3.639917,2.149005,-279.616804
3,3,Juno,58600.0,2.669150,0.256942,0.226699,2.964490,4.330836,0.609557,5.33,0.32,JPL 108,0.996719,1592.787270,0.003945,1.621697,2.008860,1.012141,-154.522558
4,4,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,3.20,0.32,JPL 34,-4.436417,1325.432768,0.004740,6.115656,0.006132,4.442550,-352.940421
5,5,Astraea,58600.0,2.574249,0.191095,0.093672,2.470978,6.260280,4.928221,6.85,0.15,JPL 108,-1.738676,1508.600442,0.004165,1.093108,0.709396,2.448072,325.328481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541124,541124,2018 RP23,58600.0,2.586399,0.289358,0.088749,2.000720,3.913328,1.075531,17.30,0.15,JPL 7,1.654537,1519.293350,0.004136,0.706394,1.285400,-0.369137,-260.066715
541125,541125,2018 RV23,58600.0,3.113036,0.213678,0.203046,0.544794,0.242079,0.130760,16.10,0.15,JPL 8,0.206083,2006.201725,0.003132,0.917633,0.992956,0.786873,-41.751113
541126,541126,2018 RP24,58600.0,2.453880,0.176693,0.194504,2.649626,3.695880,0.937231,17.30,0.15,JPL 6,1.258854,1404.036362,0.004475,0.999551,1.321174,0.062320,-209.433076


In [22]:
# Index for Ceres
idx_ast = np.where(ast_elt.Name=='Ceres')
# Get the index for observation time
ts = inputs['ts'][idx_ast].flatten()
ts

array([51544., 51545., 51546., ..., 66517., 66518., 66519.], dtype=float32)

In [23]:
# Index of observation time <= obs_time
idx_t = np.searchsorted(ts, obstime_mjd)-1

In [24]:
inputs['ts'][idx_ast, idx_t]

array([[58600.]], dtype=float32)

In [25]:
q0 = outputs['q'][:, idx_t]
q1 = outputs['q'][:, idx_t+1]
q = 0.5 * q0 + 0.5 * q1
q.shape

(999, 3)

In [26]:
# Predicted position and direction of ceres
q_ast_mse = outputs['q'][idx_ast, idx_t].flatten()
np.round(q_ast_mse,6)

array([-1.356567, -2.37266 ,  0.175377], dtype=float32)

In [27]:
np.linalg.norm(q_ast_mse)

2.7387128

In [28]:
q_earth_mse = get_earth_pos(obstime_mjd)

In [29]:
np.round(q_earth_mse, 6)

array([-8.07118e-01, -6.01307e-01,  3.10000e-05])

In [30]:
q_rel_mse = q_ast_mse - q_earth_mse
np.round(q_rel_mse, 6)

array([-0.549449, -1.771353,  0.175346])

In [31]:
u_mse = q_rel_mse / np.linalg.norm(q_rel_mse)
np.round(u_mse, 6)

array([-0.294946, -0.950867,  0.094126])

**Compare JPL and MSE Direction Calculations**

In [34]:
u_diff = u_mse - u_jpl
u_diff

array([-0.00706335,  0.00224531,  0.00084465])

In [42]:
np.linalg.norm(u_diff)

0.0074596062185876